In [4]:
import re
import requests
from dataclasses import dataclass
from textwrap import dedent

import nbformat as nbf
from bs4 import BeautifulSoup


In [5]:
WEEK_NB = 2

URLS = {
    "Sum Mixed Array": "https://www.codewars.com/kata/57eaeb9578748ff92a000009/python",
    "Multiplication Table": "https://www.codewars.com/kata/5a2fd38b55519ed98f0000ce/python",
    "Numbers to Letters": "https://www.codewars.com/kata/57ebaa8f7b45ef590c00000c/python",
    "English Beggars": "https://www.codewars.com/kata/59590976838112bfea0000fa/python",
    "Find the first non-consecutive number": "https://www.codewars.com/kata/58f8a3a27a5c28d92e000144/python",
    "Enough is enough": "https://www.codewars.com/kata/554ca54ffa7d91b236000023/python",
    "I love you, a little , a lot, passionately ... not at all": "https://www.codewars.com/kata/57f24e6a18e9fad8eb000296/python",
    "Filter Long Words": "https://www.codewars.com/kata/5697fb83f41965761f000052/python",
    "Populate hash with array keys and default value": "https://www.codewars.com/kata/51c38e14ea1c97ffaf000003/python",
}

In [6]:
# for title, url in URLS.items():
#     response = requests.get(url)
#     print(f"---{title}---")
#     ranks = list(re.findall(r'(rank="-\d+"|\d+ kyu)', response.content.decode()))
#     print(ranks)

In [7]:
@dataclass
class Challenge:
    title: str
    level: int
    descr: str

def get_challenge(url):
    response = requests.get(url)
    if response.status_code != 200:
        raise Exception(f"Error fetching page: {response}")

    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.title.text.replace(" | Codewars", "")

    content = response.content.decode()
    m = re.search(r'\\"description\\":\\"(.*?)\\"', content, flags=re.S)
    descr = m.group(1).replace("\\\\n", "\n")

    # there might be some content specific to code: ```if:<language> or ```if-not:<language>
    descr = re.sub(r"^```if:(?!python)\w+.*?^```", "", descr, flags=re.S|re.M)
    descr = re.sub(r"^```if-not:python.*?^```", "", descr, flags=re.S|re.M)

    level = re.search(r"(\d) kyu", content).group(1)
    
    return Challenge(title, int(level), descr)


In [8]:
# print(get_description("https://www.codewars.com/kata/5a2fd38b55519ed98f0000ce/python"))

In [9]:
LEVELS = {
    8:"❽",
    7:"❼",
    6:"❻"
}


def iter_challenges(week_nb, urls):
    header = dedent(f"""\
        # ⇝ Coding 💻 Challenges Week {week_nb} ⇜

        ![unpackAI Logo](../images/unpackAI_logo_whiteBG.svg)
            
            
        **NOTE**: in the solution, the code assert function(...) == value is used to test the solution.
        ... so you don't have to care about it
    """)
    yield nbf.v4.new_markdown_cell(header)

    for url in urls:
        challenge = get_challenge(url)
        print(f"* {challenge.title} ({challenge.level})")
        train_url = re.sub(f"(/train)?/python$", "", url) + "/train/python"

        title = dedent(f"""\
        # {challenge.title} {LEVELS[challenge.level]}
        🔗{train_url}
        """)
        yield nbf.v4.new_markdown_cell(f"{title}\n{challenge.descr}")

        yield nbf.v4.new_code_cell("# TODO: add solution")


In [10]:
fname = f'challenges_W{WEEK_NB:02d}_TMP.ipynb'
print(f"Starting creation of Notebook {fname}:")

notebook = nbf.v4.new_notebook()
notebook['cells'] = list(iter_challenges(WEEK_NB, URLS.values()))

with open(fname, 'w', encoding="utf-8") as f:
    nbf.write(notebook, f)
print("=> SUCCESSFUL!")

Starting creation of Notebook challenges_W02.ipynb:
* Sum Mixed Array (8)
* Multiplication table for number (8)
* Numbers to Letters (7)
* English beggars (6)
* Find the first non-consecutive number (8)
* Delete occurrences of an element if it occurs more than n times (6)
* I love you,  a little ,  a lot,  passionately ... not at all (8)
* Filter Long Words (7)
* Populate hash with array keys and default value (7)
=> SUCCESSFUL!
